In [1]:
import numpy as np
import pandas as pd
import os
# %pip install arbitrageRepair
from arbitragerepair import constraints, repair
# %pip install pandas_market_calendars
import pandas_market_calendars as mcal

Note: you may need to restart the kernel to use updated packages.


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
raw_data = pd.read_csv('Call Option Data/fqjo3s8eacwzxkcw.csv')
tickers = raw_data['ticker'].unique()

In [4]:
tickers

array(['AMZN', 'JPM', 'JNJ', 'MSFT', 'PG', 'WMT', 'GOOGL', 'V', 'TSLA'],
      dtype=object)

In [5]:
raw_data.head()

,date,symbol,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,impl_volatility,optionid,forward_price,ticker,index_flag,issuer,exercise_style
0,2018-01-02,AMZN 180105C1000000,2018-01-05,2017-12-29,C,1000000,189.35,190.6,0,0.962943,118189007,NaN,AMZN,0,AMAZON.COM INC.,A
1,2018-01-02,AMZN 180105C1002500,2018-01-05,NaN,C,1002500,186.85,188.1,0,0.951017,118541021,NaN,AMZN,0,AMAZON.COM INC.,A
2,2018-01-02,AMZN 180105C1005000,2018-01-05,NaN,C,1005000,184.35,185.6,0,0.939104,118541022,NaN,AMZN,0,AMAZON.COM INC.,A
3,2018-01-02,AMZN 180105C1007500,2018-01-05,NaN,C,1007500,181.85,183.1,0,0.927205,118541023,NaN,AMZN,0,AMAZON.COM INC.,A
4,2018-01-02,AMZN 180105C1010000,2018-01-05,2017-12-29,C,1010000,179.35,180.6,0,0.915319,118189008,NaN,AMZN,0,AMAZON.COM INC.,A


In [6]:
# Arbitrage repair

def arbitrageRepair(columnT, columnK, columnC_ask, columnC_bid, columnF):
    # normalise strikes and call prices
    normaliser = constraints.Normalise()
    T = np.array(columnT)
    K = np.array(columnK)
    C_bid = np.array(columnC_bid)
    C_ask = np.array(columnC_ask)
    C = (C_ask+C_bid)/2
    F = np.array(columnF)
    normaliser.fit(T, K, C, F)
    T1, K1, C1 = normaliser.transform(T, K, C)
    _, _, C1_bid = normaliser.transform(T, K, C_bid)
    _, _, C1_ask = normaliser.transform(T, K, C_ask)

    # construct arbitrage constraints and detect violation
    mat_A, vec_b, _, _ = constraints.detect(T1, K1, C1, verbose=False)

    epsilon = repair.l1(mat_A, vec_b, C1)

    # de-normalise
    K0, C0_ask = normaliser.inverse_transform(K1, C1_ask + epsilon)
    _, C0_bid = normaliser.inverse_transform(K1, C1_bid + epsilon)

    return K0, C0_ask, C0_bid

In [7]:
# Data processing
## call and put options, ask and bid prices

def dataProcess2Di(df,t1,t2,ticker,stock_daily):

    # call options data
    df_t1_C = df.loc[(df.exdate==t1)&(df.cp_flag=='C'), ['date','exdate','cp_flag','strike_price','volume','best_offer','best_bid','impl_volatility']]
    df_t2_C = df.loc[(df.exdate==t2)&(df.cp_flag=='C'), ['date','exdate','cp_flag','strike_price','volume','best_offer','best_bid','impl_volatility']]

    # put options data
    df_t1_P = df.loc[(df.exdate==t1)&(df.cp_flag=='P'), ['date','exdate','cp_flag','strike_price','volume','best_offer','best_bid','impl_volatility']]
    df_t2_P = df.loc[(df.exdate==t2)&(df.cp_flag=='P'), ['date','exdate','cp_flag','strike_price','volume','best_offer','best_bid','impl_volatility']]

    t0List = list(df_t1_C.date.unique())

    # Threshold for trading volume
    volumeThreshold = 5
    
    result = pd.DataFrame()
    for t0 in t0List:
        ## Check data availability at both T1 and T2:
        if len(df_t1_C[df_t1_C.date==t0])*len(df_t2_C[df_t2_C.date==t0])==0:
            continue
    
        ## T1:
        # t1: call options with 10 largest trading volumes above the threshold
        tmp_t1_C = df_t1_C[df_t1_C.date==t0].sort_values('volume',ascending=False).iloc[0:10]
        df_t0t1_C = tmp_t1_C[tmp_t1_C.volume >= volumeThreshold].reset_index(drop=True)
        # t1: put options with 10 largest trading volumes above the threshold
        tmp_t1_P = df_t1_P[df_t1_P.date==t0].sort_values('volume',ascending=False).iloc[0:10]
        df_t0t1_P = tmp_t1_P[tmp_t1_P.volume >= volumeThreshold].reset_index(drop=True)
        # Available both call and put options at t1?
        if len(df_t0t1_C)*len(df_t0t1_P)==0:
            continue

        # merge with put/call options data and stock price data
        df_t0t1_C = df_t0t1_C.merge(df_t1_P, on = ['date','exdate','strike_price'], suffixes = ('_C','_P'))
        df_t0t1_C = pd.merge(df_t0t1_C, stock_daily, on = 'date')
        
        df_t0t1_P = df_t0t1_P.merge(df_t1_C, on = ['date','exdate','strike_price'], suffixes = ('_P','_C'))
        df_t0t1_P = pd.merge(df_t0t1_P, stock_daily, on = 'date')

        # calculate the forward price:
        df_t0t1_C['option_price_C'] = df_t0t1_C[['best_offer_C','best_bid_C']].mean(axis=1)
        df_t0t1_C['option_price_P'] = df_t0t1_C[['best_offer_P','best_bid_P']].mean(axis=1)
        df_t0t1_C['put_spread'] = df_t0t1_C['best_offer_P'] - df_t0t1_C['best_bid_P']
        df_t0t1_C['forward_price'] = df_t0t1_C['adjusted_price']/((df_t0t1_C['adjusted_price']-(df_t0t1_C['option_price_C']-df_t0t1_C['option_price_P']))/df_t0t1_C['strike_price'])

        df_t0t1_P['option_price_C'] = df_t0t1_P[['best_offer_C','best_bid_C']].mean(axis=1)
        df_t0t1_P['option_price_P'] = df_t0t1_P[['best_offer_P','best_bid_P']].mean(axis=1)
        df_t0t1_P['put_spread'] = df_t0t1_P['best_offer_P'] - df_t0t1_P['best_bid_P']
        df_t0t1_P['forward_price'] = df_t0t1_P['adjusted_price']/((df_t0t1_P['adjusted_price']-(df_t0t1_P['option_price_C']-df_t0t1_P['option_price_P']))/df_t0t1_P['strike_price'])

        # calculate the expiry:
        # Amazon (AMZN) is listed on the NASDAQ exchange
        amzn_t = mcal.get_calendar('NASDAQ')
        trading_days_1 = amzn_t.valid_days(start_date=t0, end_date=t1)
        df_t0t1_C['expiry'] = len(trading_days_1) / 252
        df_t0t1_P['expiry'] = len(trading_days_1) / 252

        # arbitrage repair:
        K0_t1_C, C0_ask_t1, C0_bid_t1 = arbitrageRepair(df_t0t1_C['expiry'], df_t0t1_C['strike_price'], df_t0t1_C['best_offer_C'], df_t0t1_C['best_bid_C'], df_t0t1_C['forward_price'])
        
        K0_t1_P, C0_ask_t1_P, C0_bid_t1_P = arbitrageRepair(df_t0t1_P['expiry'], df_t0t1_P['strike_price'], df_t0t1_P['best_offer_C'], df_t0t1_P['best_bid_C'], df_t0t1_P['forward_price'])
        P0_mid_t1 = (C0_ask_t1_P+C0_bid_t1_P)/2-df_t0t1_P['adjusted_price']+df_t0t1_P['strike_price']/df_t0t1_P['forward_price']*df_t0t1_P['adjusted_price']
        P0_ask_t1 = P0_mid_t1+df_t0t1_P['put_spread']/2
        P0_bid_t1 = P0_mid_t1-df_t0t1_P['put_spread']/2

        # temporary result at t0:
        result_t1_C = df_t0t1_C[['date','exdate']].copy()
        result_t1_C['cp_flag'] = 'C'
        result_t1_C['strike_price'] = K0_t1_C
        result_t1_C['ask_price'] = C0_ask_t1
        result_t1_C['bid_price'] = C0_bid_t1
        
        result_t1_P = df_t0t1_P[['date','exdate']].copy()
        result_t1_P['cp_flag'] = 'P'
        result_t1_P['strike_price'] = K0_t1_P
        result_t1_P['ask_price'] = P0_ask_t1
        result_t1_P['bid_price'] = P0_bid_t1
        

        ## T2:
        # t2: call options with 10 largest trading volumes above the threshold
        tmp_t2_C = df_t2_C[df_t2_C.date==t0].sort_values('volume',ascending=False).iloc[0:10]
        df_t0t2_C = tmp_t2_C[tmp_t2_C.volume >= volumeThreshold].reset_index(drop=True)
        # t2: put options with 10 largest trading volumes above the threshold
        tmp_t2_P = df_t2_P[df_t2_P.date==t0].sort_values('volume',ascending=False).iloc[0:10]
        df_t0t2_P = tmp_t2_P[tmp_t2_P.volume >= volumeThreshold].reset_index(drop=True)
        # Available both call and put options at t2?
        if len(df_t0t2_C)*len(df_t0t2_P)==0:
            continue

        # merge with put/call options data and stock price data
        df_t0t2_C = df_t0t2_C.merge(df_t2_P, on = ['date','exdate','strike_price'], suffixes = ('_C','_P'))
        df_t0t2_C = pd.merge(df_t0t2_C, stock_daily, on = 'date')
        
        df_t0t2_P = df_t0t2_P.merge(df_t2_C, on = ['date','exdate','strike_price'], suffixes = ('_P','_C'))
        df_t0t2_P = pd.merge(df_t0t2_P, stock_daily, on = 'date')

        # calculate the forward price:
        df_t0t2_C['option_price_C'] = df_t0t2_C[['best_offer_C','best_bid_C']].mean(axis=1)
        df_t0t2_C['option_price_P'] = df_t0t2_C[['best_offer_P','best_bid_P']].mean(axis=1)
        df_t0t2_C['put_spread'] = df_t0t2_C['best_offer_P'] - df_t0t2_C['best_bid_P']
        df_t0t2_C['forward_price'] = df_t0t2_C['adjusted_price']/((df_t0t2_C['adjusted_price']-(df_t0t2_C['option_price_C']-df_t0t2_C['option_price_P']))/df_t0t2_C['strike_price'])

        df_t0t2_P['option_price_C'] = df_t0t2_P[['best_offer_C','best_bid_C']].mean(axis=1)
        df_t0t2_P['option_price_P'] = df_t0t2_P[['best_offer_P','best_bid_P']].mean(axis=1)
        df_t0t2_P['put_spread'] = df_t0t2_P['best_offer_P'] - df_t0t2_P['best_bid_P']
        df_t0t2_P['forward_price'] = df_t0t2_P['adjusted_price']/((df_t0t2_P['adjusted_price']-(df_t0t2_P['option_price_C']-df_t0t2_P['option_price_P']))/df_t0t2_P['strike_price'])

        # calculate the expiry:
        trading_days_2 = amzn_t.valid_days(start_date=t0, end_date=t2)
        df_t0t2_C['expiry'] = len(trading_days_2) / 252
        df_t0t2_P['expiry'] = len(trading_days_2) / 252

        # arbitrage repair:
        K0_t2_C, C0_ask_t2, C0_bid_t2 = arbitrageRepair(df_t0t2_C['expiry'], df_t0t2_C['strike_price'], df_t0t2_C['best_offer_C'], df_t0t2_C['best_bid_C'], df_t0t2_C['forward_price'])
        
        K0_t2_P, C0_ask_t2_P, C0_bid_t2_P = arbitrageRepair(df_t0t2_P['expiry'], df_t0t2_P['strike_price'], df_t0t2_P['best_offer_C'], df_t0t2_P['best_bid_C'], df_t0t2_P['forward_price'])
        P0_mid_t2 = (C0_ask_t2_P+C0_bid_t2_P)/2-df_t0t2_P['adjusted_price']+df_t0t2_P['strike_price']/df_t0t2_P['forward_price']*df_t0t2_P['adjusted_price']
        P0_ask_t2 = P0_mid_t2+df_t0t2_P['put_spread']/2
        P0_bid_t2 = P0_mid_t2-df_t0t2_P['put_spread']/2

        # temporary result at t0:
        result_t2_C = df_t0t2_C[['date','exdate']].copy()
        result_t2_C['cp_flag'] = 'C'
        result_t2_C['strike_price'] = K0_t2_C
        result_t2_C['ask_price'] = C0_ask_t2
        result_t2_C['bid_price'] = C0_bid_t2
        
        result_t2_P = df_t0t2_P[['date','exdate']].copy()
        result_t2_P['cp_flag'] = 'P'
        result_t2_P['strike_price'] = K0_t2_P
        result_t2_P['ask_price'] = P0_ask_t2
        result_t2_P['bid_price'] = P0_bid_t2
        

        ## Concat horizontally
        result_t1_C = result_t1_C.reset_index(drop=True)
        result_t2_C = result_t2_C.reset_index(drop=True)
        result_C = pd.concat([result_t1_C, result_t2_C.drop(columns=['date','cp_flag'])], axis=1)
        result_t1_P = result_t1_P.reset_index(drop=True)
        result_t2_P = result_t2_P.reset_index(drop=True)
        result_P = pd.concat([result_t1_P, result_t2_P.drop(columns=['date','cp_flag'])], axis=1)
        result_tmp = pd.concat([result_C,result_P], axis=0, ignore_index=True)
        
        ## Concat vertically
        result = pd.concat([result,result_tmp], axis=0, ignore_index=True)

    if len(result) != 0:
        result = result.reset_index(drop=True)
        result = pd.merge(result, stock_daily, on = 'date')
        result.columns = ['t0', 'T1', 'cp_flag', 'K1',  'Pi1_ask', 'Pi1_bid', 'T2', 'K2', 'Pi2_ask', 'Pi2_bid', 'Adj_S0']
        file_name = 'data/options_callput_askbid/{}_{}_{}.csv'.format(ticker,pd.to_datetime(t1).strftime('%Y%m%d'),pd.to_datetime(t2).strftime('%Y%m%d'))
        result.to_csv(file_name, header=True)

In [8]:
# Data of adjusted stocks prices
stocks = pd.read_csv('adjusted_stocks.csv')
stocks = stocks.rename(columns = {'Date':'date'})
stocks.head()

,date,AMZN,JPM,JNJ,MSFT,PG,WMT,GOOGL,V,TSLA
0,2018-01-02,1189.010010,90.125488,117.502670,80.080925,76.669670,88.322428,1073.209991,109.733635,320.530014
1,2018-01-03,1204.199982,90.217331,118.625122,80.453613,76.576607,89.092855,1091.520004,110.826103,317.249994
2,2018-01-04,1209.589996,91.509758,118.616730,81.161705,77.117928,89.173491,1095.759964,111.238144,314.619999
3,2018-01-05,1229.140015,90.922310,119.595680,82.167976,77.168678,89.702036,1110.289993,113.902199,316.580000
4,2018-01-08,1246.869965,91.056587,119.747589,82.251801,77.574646,91.027891,1114.209976,114.362175,336.410007


In [9]:
# path
os.makedirs('data/options_callput_askbid/')

In [ ]:
for ticker in tickers:
    # stock_Daily = pd.read_csv('data/adjusted_Stock_Daily/{}_stock_daily_adjusted.csv'.format(ticker.lower()),index_col=0)
    stock_Daily = stocks[['date', ticker]]
    stock_Daily = stock_Daily.rename(columns = {ticker:'adjusted_price'})
    stock_Daily['date'] = pd.to_datetime(stock_Daily['date'])
    
    df = raw_data.loc[raw_data.ticker==ticker,
                      ['date','exdate','cp_flag','strike_price','best_bid','best_offer','volume','impl_volatility']].copy()
    df['date'] = pd.to_datetime(df['date'])
    df['exdate'] = pd.to_datetime(df['exdate'])
    df['strike_price'] = df['strike_price']/1000 # scaling strike

    ## Select T1 and T2 which has closest distance of 1 month
    T1_List = pd.to_datetime(df['exdate'].unique())
    T1_T2_List = pd.DataFrame()
    for t1 in T1_List:
        t2 = T1_List[T1_List.year*12+T1_List.month == t1.year*12+t1.month+1]
        if len(t2) > 1:
            t2 = t2[t2.day == min(np.abs(t2.day-t1.day))]
        if len(t2) > 0:
            T1_T2_List = pd.concat([T1_T2_List,pd.DataFrame([t1,t2[0]]).T])
    for t2 in T1_T2_List[1]:
        if len(T1_T2_List[T1_T2_List[1]==t2]) >1:
            df_t2 = T1_T2_List[T1_T2_List[1]==t2]
            df_t2['diff'] = np.abs(df_t2[0].dt.day - t2.day)
            df_t2 = df_t2.sort_values(by='diff').iloc[:1,:2]
            T1_T2_List = pd.concat([T1_T2_List[T1_T2_List[1]!=t2],df_t2])

    ## Data processing
    for i in range(len(T1_T2_List)):
        t1 = T1_T2_List.iloc[i,0]
        t2 = T1_T2_List.iloc[i,1]
        dataProcess2Di(df,t1,t2,ticker=ticker,stock_daily=stock_Daily)

GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
      0: obj =   0.000000000e+00 inf =   2.259e-02 (1)
      2: obj =   5.203529393e-05 inf =   0.000e+00 (0)
*    20: obj =   5.203529393e-05 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
      0: obj =   0.000000000e+00 inf =   2.161e-02 (1)
      2: obj =   4.343032537e-05 inf =   0.000e+00 (0)
*    20: obj =   4.343032537e-05 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Simplex Optimizer, v4.65
25 rows, 12 columns, 66 non-zeros
      0: obj =   0.000000000e+00 inf =   3.780e-01 (2)
      4: obj =   9.934343574e-04 inf =   0.000e+00 (0)
*    12: obj =   8.925253107e-04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Simplex Optimizer, v4.65
9 rows, 4 columns, 18 non-zeros
*     0: obj =   0.000000000e+00 inf =   0.000e+00 (4)
*     4: obj =   0.000000000e+00 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Simplex Optimizer, v4.65
41 ro

GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
*     0: obj =   0.000000000e+00 inf =   0.000e+00 (20)
*    20: obj =   0.000000000e+00 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
      0: obj =   0.000000000e+00 inf =   1.322e+00 (3)
      4: obj =   1.566338071e-02 inf =   0.000e+00 (0)
*    20: obj =   1.566338071e-02 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
      0: obj =   0.000000000e+00 inf =   4.350e-01 (3)
      5: obj =   5.950010914e-03 inf =   0.000e+00 (0)
*    19: obj =   5.950010914e-03 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
      0: obj =   0.000000000e+00 inf =   5.489e-01 (2)
      5: obj =   5.816189798e-03 inf =   0.000e+00 (0)
*    20: obj =   5.816189798e-03 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Simplex Optimizer, v4.65


GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
      0: obj =   0.000000000e+00 inf =   1.982e+00 (3)
      5: obj =   1.379043381e-02 inf =   0.000e+00 (0)
*    20: obj =   1.379043381e-02 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
      0: obj =   0.000000000e+00 inf =   9.122e-01 (5)
     10: obj =   1.673694278e-02 inf =   0.000e+00 (0)
*    21: obj =   1.648504311e-02 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
      0: obj =   0.000000000e+00 inf =   9.051e-01 (3)
      6: obj =   1.031920524e-02 inf =   0.000e+00 (0)
*    20: obj =   1.031920524e-02 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
      0: obj =   0.000000000e+00 inf =   3.179e+00 (3)
      7: obj =   2.435928916e-02 inf =   0.000e+00 (0)
*    20: obj =   2.435928916e-02 inf =   0.000e+00 (0)
OP

      0: obj =   0.000000000e+00 inf =   2.884e+00 (5)
      9: obj =   2.655061893e-02 inf =   0.000e+00 (0)
*    21: obj =   2.552926850e-02 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
      0: obj =   0.000000000e+00 inf =   1.926e+00 (3)
      7: obj =   1.825921567e-02 inf =   0.000e+00 (0)
*    21: obj =   1.772088415e-02 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
      0: obj =   0.000000000e+00 inf =   8.276e-01 (2)
      4: obj =   5.171776035e-03 inf =   0.000e+00 (0)
*    20: obj =   5.099417457e-03 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
      0: obj =   0.000000000e+00 inf =   3.369e-02 (1)
      2: obj =   3.578442405e-04 inf =   0.000e+00 (0)
*    20: obj =   3.578442405e-04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Simplex Optimizer, v4.65
41 rows, 20

GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
      0: obj =   0.000000000e+00 inf =   1.114e+00 (4)
      8: obj =   1.715891816e-02 inf =   0.000e+00 (0)
*    21: obj =   1.715891816e-02 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
      0: obj =   0.000000000e+00 inf =   1.184e+00 (5)
     15: obj =   4.886378153e-02 inf =   0.000e+00 (0)
*    24: obj =   4.871671664e-02 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
      0: obj =   0.000000000e+00 inf =   7.567e-01 (4)
      8: obj =   7.868912354e-03 inf =   0.000e+00 (0)
*    21: obj =   7.868912354e-03 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
      0: obj =   0.000000000e+00 inf =   7.203e+00 (5)
     13: obj =   5.705456402e-02 inf =   0.000e+00 (0)
*    23: obj =   5.682674087e-02 inf =   0.000e+00 (0)
OP

...

GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
      0: obj =   0.000000000e+00 inf =   4.039e-01 (2)
      4: obj =   6.475107742e-03 inf =   0.000e+00 (0)
*    20: obj =   6.475107742e-03 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
      0: obj =   0.000000000e+00 inf =   6.059e+00 (5)
      9: obj =   2.807407917e-02 inf =   0.000e+00 (0)
*    22: obj =   2.807407917e-02 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
      0: obj =   0.000000000e+00 inf =   5.868e-01 (2)
      4: obj =   5.428985205e-03 inf =   0.000e+00 (0)
*    20: obj =   5.428985205e-03 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
      0: obj =   0.000000000e+00 inf =   9.185e+00 (4)
     12: obj =   6.509711525e-02 inf =   0.000e+00 (0)
*    21: obj =   6.509711525e-02 inf =   0.000e+00 (0)
OP

GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
      0: obj =   0.000000000e+00 inf =   1.129e+00 (4)
      8: obj =   7.910265177e-03 inf =   1.735e-17 (0)
*    20: obj =   7.910265177e-03 inf =   1.735e-17 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
      0: obj =   0.000000000e+00 inf =   1.031e+00 (5)
     10: obj =   9.221565694e-03 inf =   0.000e+00 (0)
*    21: obj =   9.221565694e-03 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
      0: obj =   0.000000000e+00 inf =   1.255e+00 (3)
      7: obj =   9.269104799e-03 inf =   0.000e+00 (0)
*    21: obj =   9.269104799e-03 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
      0: obj =   0.000000000e+00 inf =   4.668e+00 (4)
     12: obj =   1.241961264e-02 inf =   0.000e+00 (0)
*    20: obj =   1.240788057e-02 inf =   0.000e+00 (0)
OP

GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
      0: obj =   0.000000000e+00 inf =   2.871e+00 (5)
     10: obj =   1.864780564e-02 inf =   0.000e+00 (0)
*    21: obj =   1.864780564e-02 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
      0: obj =   0.000000000e+00 inf =   1.510e+01 (4)
      7: obj =   1.712347614e-02 inf =   0.000e+00 (0)
*    21: obj =   1.712347614e-02 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
      0: obj =   0.000000000e+00 inf =   6.375e+00 (4)
      8: obj =   1.707101208e-02 inf =   0.000e+00 (0)
*    20: obj =   1.659466382e-02 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
      0: obj =   0.000000000e+00 inf =   9.965e+00 (4)
      8: obj =   1.163134209e-02 inf =   0.000e+00 (0)
*    20: obj =   1.119310629e-02 inf =   0.000e+00 (0)
OP

GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
      0: obj =   0.000000000e+00 inf =   1.098e+01 (4)
      8: obj =   1.136120635e-02 inf =   0.000e+00 (0)
*    20: obj =   7.780590633e-03 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
      0: obj =   0.000000000e+00 inf =   3.733e+01 (6)
     13: obj =   2.107606850e-02 inf =   0.000e+00 (0)
*    23: obj =   2.087799573e-02 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
      0: obj =   0.000000000e+00 inf =   1.630e+01 (6)
     11: obj =   1.438519617e-02 inf =   0.000e+00 (0)
*    20: obj =   1.297605815e-02 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
      0: obj =   0.000000000e+00 inf =   5.492e+00 (3)
      8: obj =   1.165017749e-02 inf =   0.000e+00 (0)
*    20: obj =   1.155073880e-02 inf =   0.000e+00 (0)
OP

GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
      0: obj =   0.000000000e+00 inf =   7.291e-01 (4)
      7: obj =   2.495710503e-03 inf =   0.000e+00 (0)
*    21: obj =   2.495710503e-03 inf =   2.776e-17 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
      0: obj =   0.000000000e+00 inf =   1.533e+00 (4)
      8: obj =   8.937077335e-03 inf =   2.765e-16 (0)
*    22: obj =   8.625109506e-03 inf =   2.220e-16 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
      0: obj =   0.000000000e+00 inf =   2.406e+00 (6)
     10: obj =   2.010694203e-02 inf =   0.000e+00 (0)
*    21: obj =   2.004669741e-02 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
GLPK Simplex Optimizer, v4.65
41 rows, 20 columns, 114 non-zeros
      0: obj =   0.000000000e+00 inf =   9.893e+00 (3)
     12: obj =   3.915976609e-02 inf =   0.000e+00 (0)
*    21: obj =   3.311689528e-02 inf =   0.000e+00 (0)
OP